## Creating function: &thinsp; convective object properties

In [ ]:
# scene to calculate index from
pr_day = ds.pr4_days.isel(time=0)
pr_day3d = np.expand_dims(pr_day,axis=2) # used for broadcasting
lat = pr_day.lat
lon = pr_day.lon
conv_threshold = ds_percentile.pr_97.mean(dim=('time'))
L = skm.label(pr_day.where(pr_day>=conv_threshold,0)>0, background=0,connectivity=2)
myFuncs.connect_boundary(L)
labels = np.unique(L)[1:]

In [ ]:
# dimensions used for calculation
lat = pr_day.lat.data
lon = pr_day.lon.data
lonm,latm = np.meshgrid(lon,lat)
dlat = (lat[1]-lat[0])
dlon = (lon[1]-lon[0])
R = 6371
aream = np.cos(np.deg2rad(latm))*np.float64(dlon*dlat*R**2*(np.pi/180)**2)

latm3d = np.expand_dims(latm,axis=2) # used for broadcasting
lonm3d = np.expand_dims(lonm,axis=2)
aream3d = np.expand_dims(aream,axis=2) 
shape_L = np.shape(L)

In [ ]:
# number index and area fraction of a scene
numberIndex_scene = len(labels)
conv_day = (pr_day.where(pr_day>=conv_threshold,0)>0)*1
areaf_scene = np.sum(conv_day * aream)/np.sum(aream)

# aWeights = np.cos(np.deg2rad(lat))
# areaf_scene = conv_day.weighted(aWeights).mean(dim=('lat','lon'))

In [ ]:
np.float32(areaf_scene)

0.028038016

In [ ]:
# for multiple scenes as function
def calc_numberIndex(precip, listOfdays, conv_threshold):
    numberIndex, areaf = [], []

    lat = precip.lat.data
    lon = precip.lon.data
    lonm,latm = np.meshgrid(lon,lat)
    dlat = (lat[1]-lat[0])
    dlon = (lon[1]-lon[0])
    R = 6371
    aream = np.cos(np.deg2rad(latm))*np.float64(dlon*dlat*R**2*(np.pi/180)**2)


    for day in listOfdays:
        pr_day = precip.isel(time=day)
        L = skm.label(pr_day.where(pr_day>=conv_threshold,0)>0, background=0,connectivity=2)
        myFuncs.connect_boundary(L)
        labels = np.unique(L)[1:]

        numberIndex_scene = len(labels)
        
        conv_day = (pr_day.where(pr_day>=conv_threshold,0)>0)*1
        areaf_scene = np.sum(conv_day * aream)/np.sum(aream)
        
        numberIndex = np.append(numberIndex, numberIndex_scene)
        areaf = np.append(areaf, areaf_scene)


    numberIndex = xr.DataArray(
        data=numberIndex,
        dims=['time'],
        coords={'time': precip.time.data[0:len(listOfdays)]},
        attrs={'units':'Nb'}
        )

    areaf = xr.DataArray(
        data=areaf,
        dims=['time'],
        coords={'time': precip.time.data[0:len(listOfdays)]}
        )

    return numberIndex, areaf
    

In [ ]:
# calling function on data array
listOfdays = np.arange(0,len(ds.pr4_days.time))
precip = ds.pr4_days

numberIndex, areaf = calc_numberIndex(precip, listOfdays, conv_threshold)

In [ ]:
# precipitation and area of objects in a scene
obj3d = np.stack([(L==label) for label in labels],axis=2)*1 # 3d matrix with each object in a scene being a binary 2d slice
o_prScene = np.sum(obj3d * pr_day3d * aream3d, axis=(0,1)) / np.sum(obj3d * aream3d, axis=(0,1))
o_areaScene = np.sum(obj3d * aream3d, axis=(0,1))

In [ ]:
print(len(o_prScene))
print(type(o_prScene))
print(o_prScene)
print(len(o_areaScene))
print(o_areaScene)

15
<class 'numpy.ndarray'>
[29.00566864 18.44254684 27.20878045 21.94720348 23.34866253 22.47823334
 21.92810917 31.64316842 17.9397316  27.04435205 26.63710974 22.7985008
 18.94947624 18.25418472 24.99199563]
15
[ 169251.97805239   84625.9890262   466319.92944535 1232550.47127691
  475431.01125242  189440.47633393  189440.47633393 1833366.03003652
   96782.87117449  388051.84548862  581157.40744228  864156.77710563
  378880.95266786   93580.67721021  258301.67645397]


In [ ]:
# as function
def calc_area_pr(precip, listOfdays, conv_threshold):
    o_pr, o_area = [], []

    lat = precip.lat.data
    lon = precip.lon.data
    lonm,latm = np.meshgrid(lon,lat)
    dlat = (lat[1]-lat[0])
    dlon = (lon[1]-lon[0])
    R = 6371
    aream = np.cos(np.deg2rad(latm))*np.float64(dlon*dlat*R**2*(np.pi/180)**2)

    latm3d = np.expand_dims(latm,axis=2) # used for broadcasting
    lonm3d = np.expand_dims(lonm,axis=2)
    aream3d = np.expand_dims(aream,axis=2)

    for day in listOfdays:
        pr_day = precip.isel(time=day)
        pr_day3d = np.expand_dims(pr_day,axis=2)
        L = skm.label(pr_day.where(pr_day>=conv_threshold,0)>0, background=0,connectivity=2)
        myFuncs.connect_boundary(L)
        labels = np.unique(L)[1:]
        obj3d = np.stack([(L==label) for label in labels],axis=2)*1 # 3d matrix with each object in a scene being a binary 2d slice
        
        o_areaScene = np.sum(obj3d * aream3d, axis=(0,1))
        o_prScene = np.sum(obj3d * pr_day3d * aream3d, axis=(0,1)) / np.sum(obj3d * aream3d, axis=(0,1))
        
        o_area = np.append(o_area, o_areaScene)
        o_pr = np.append(o_pr, o_prScene)
        
    o_area = xr.DataArray(
        data=o_area,
        attrs={'units':'km^2'}
        )

    o_pr = xr.DataArray(
        data=o_pr,
        attrs={'descrption': 'area weighted precipitation in contiguous convective region',
                'units':'mm/day'}
        )

    return o_area, o_pr


In [ ]:
# calling function on data array
listOfdays = np.arange(0,len(ds.pr4_days.time))
precip = ds.pr4_days

o_area, o_pr = calc_area_pr(precip, listOfdays, conv_threshold)

In [ ]:
print(len(o_pr))
print(o_pr)
print(len(o_area))
print(o_area)

50
<xarray.DataArray (dim_0: 50)>
array([29.00566864, 18.44254684, 27.20878045, 21.94720348, 23.34866253,
       22.47823334, 21.92810917, 31.64316842, 17.9397316 , 27.04435205,
       26.63710974, 22.7985008 , 18.94947624, 18.25418472, 24.99199563,
       24.27660839, 22.33764331, 24.31245613, 27.70028887, 19.06106186,
       40.98479928, 21.58437347, 25.00776037, 21.85125542, 21.08172798,
       23.2420556 , 27.49645519, 24.09871721, 25.64299393, 23.54604103,
       20.8414402 , 18.41514946, 19.3091373 , 52.54117617, 22.41043308,
       19.64092064, 24.80706503, 21.44767491, 21.03659439, 28.72734116,
       20.50912885, 19.69573784, 20.81357612, 38.88082882, 19.38501358,
       25.97257169, 23.51097551, 25.13835216, 20.80052754, 19.14166069])
Dimensions without coordinates: dim_0
Attributes:
    descrption:  area weighted precipitation in contiguous convective region
    units:       mm/day
50
<xarray.DataArray (dim_0: 50)>
array([ 169251.97805239,   84625.9890262 ,  466319.92944535,

## on larger dataset

In [ ]:
time_range = pd.date_range("1970/01/01","2000/01/01",freq='D',inclusive='left') # 30 years of daily data

pr_day = ds.pr4_days.isel(time=0)
pr_day = np.expand_dims(pr_day, axis=0)
array = np.ones(shape = (len(time_range), len(ds.pr4_days.lat), len(ds.pr4_days.lon)))
pr_big = array * pr_day 
np.shape(pr_big)

(10957, 22, 128)

In [ ]:
pr_big = xr.DataArray(
                    data=pr_big,
                    dims=['time','lat', 'lon'],
                    coords={'time': time_range, 'lat': ds.pr4_days.lat.data, 'lon': ds.pr4_days.lon.data}
)

In [ ]:
len(pr_big.time)

10957

In [ ]:
listOfdays = np.arange(0,len(pr_big.time))
numberIndex, area_frac = aggFuncs.calc_numberIndex(pr_big, listOfdays, conv_threshold)

In [ ]:
print(len(number_index))
print(len(area_frac))

10957
10957


In [ ]:
area_frac[1:5]

<xarray.DataArray (time: 4)>
array([0.02803802, 0.02803802, 0.02803802, 0.02803802])
Coordinates:
  * time     (time) datetime64[ns] 1970-01-02 1970-01-03 1970-01-04 1970-01-05

In [ ]:
np.float32(conv_threshold)

17.881668